# Semantic QA over tables

https://docs.llamaindex.ai/en/stable/examples/query_engine/sec_tables/tesla_10q_table.html#extract-elements

In [41]:
from pydantic import BaseModel
from unstructured.partition.html import partition_html
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [42]:
# load pdfs
from llama_index import SimpleDirectoryReader
from llama_index.readers.file.flat_reader import FlatReader

company = "apple"

if company == "apple":
    reader = SimpleDirectoryReader(
        input_files=["../data/apple/AAPL.pdf"]
    )
    docs = reader.load_data()


elif company =="meta":
    reader = SimpleDirectoryReader(
        input_files=["../data/meta/meta.pdf"]
    )

    docs = reader.load_data()

# reader  = FlatReader()
# docs1 = reader.load_data(Path("../data/apple/AAPL.pdf"))

Task exception was never retrieved
future: <Task finished name='Task-1' coro=<run_async_tasks.<locals>._gather() done, defined at /Users/vishwasgowda/Desktop/Projects/finsight/venv/lib/python3.11/site-packages/llama_index/async_utils.py:35> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/Users/vishwasgowda/Desktop/Projects/finsight/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/tk/9mc5gz114tv5861ps8xb77nr0000gn/T/ipykernel_2161/2290914290.py", line 5, in <module>
    response = sub_query_engine.query(query)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vishwasgowda/Desktop/Projects/finsight/venv/lib/python3.11/site-packages/llama_index/indices/query/base.py", line 23, in query
    return self._query(str_or_query_bundle)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/vishwasgowda/Desktop/Projects/finsight/venv/lib/py

In [43]:
from llama_index.node_parser import UnstructuredElementNodeParser

node_parser = UnstructuredElementNodeParser()
nodes = node_parser.get_nodes_from_documents(docs)

0it [00:00, ?it/s]


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00,

In [44]:
from llama_index.retrievers import RecursiveRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
vector_retriever = vector_index.as_retriever(similarity_top_k=2)
vector_query_engine = vector_index.as_query_engine(similarity_top_k=2)

from llama_index.retrievers import RecursiveRetriever

recursive_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    verbose=True,
)
query_engine = RetrieverQueryEngine.from_args(recursive_retriever)

In [45]:
# response = query_engine.query("what was the management's discussion and analysis")
# print(str(response))

In [46]:
import os
from dotenv import load_dotenv

env_file_path = "../.env"
load_dotenv(dotenv_path=env_file_path)
api_key = os.environ.get("OPENAI_API_KEY")

In [47]:
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

from llama_index import ServiceContext
from llama_index.llms import OpenAI

import nest_asyncio

nest_asyncio.apply()

llm = OpenAI(model="gpt-3.5-turbo", api_key=api_key)
service_context = ServiceContext.from_defaults(llm=llm)

In [48]:
query_engine_tool = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name = company,
            description=f"provides information about {company} financials for the year 2022",
        ),
    )
]

sub_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tool,
    service_context=service_context,
    use_async=True
)

In [49]:
query = """
what are the risks faced in day to day operations by the company? List them out using bullet points.
"""

response = sub_query_engine.query(query)
print("-"*50)
print(str(response))

Generated 3 sub questions.
[apple] Q: What are the financial risks faced by the company?
Retrieving with query id None: What are the financial risks faced by the company?
Retrieving text node: Item 1A. Risk Factors
The Company’s business, reputation, results of operations, financial condition and stock price can be affected by a number of 
factors, whether currently known or unknown, including those described below. When any one or more of these risks materialize 
from time to time, the Company’s business, reputation, results of operations, financial condition and stock price can be materially 
and adversely affected.
Because of the following factors, as well as other factors affecting the Company’s results of operations and financial condition, 
past financial performance should not be considered to be a reliable indicator of future performance, and investors should not 
use historical trends to anticipate results or trends in future periods. This discussion of risk factors contains f

In [50]:
print(str(response))

- Frequent introductions and transitions of products and services
- Reliance on outsourcing partners for manufacturing and logistical services
- Dependence on single-source outsourcing partners for components and final assembly
- Potential disruptions in manufacturing or logistics
- Potential financial problems or disruptions in business of outsourcing partners or suppliers
- Supply and pricing risks due to industry-wide shortages and commodity pricing fluctuations
- Risks of industrial accidents at suppliers and contract manufacturers
- Potential impact of major public health issues on the global economy and supply chain
- Potential insufficiency of insurance coverage for losses
- Intense competition in global markets characterized by aggressive price competition and rapid technological change
- The need to continually introduce innovative new products and services
- The need for effective intellectual property protection
- Competition from companies with significant resources and est